___
# Ciência dos Dados - PROJETO 2

___
## Gabriel Couto


## Gabriel Miras


## Mariana Abrantes


___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [5]:
import pandas as pd
import os
import random
import numpy as np

In [8]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Gabriel Miras\Documents\INSPER\3 SEMESTRE (DP)\C DADOS DP\Projetos\Projeto 2\DP-P2


In [3]:
leitura = pd.ExcelFile('spamham2019(1).xlsx')
dados = pd.read_excel(leitura)
dados.head()

,Email,Class
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [4]:
#Separar tabela em treino e teste:
indices = range(len(dados))
indTreino = random.sample(indices, int(len(indices)*0.75))
indTeste = []
for i in indices:
    if i not in indTreino:
        indTeste.append(i)

treino = dados.loc[indTreino,:]
teste = dados.loc[indTeste,:]

#Cria índices numéricos pros dados
treino.index = range(len(treino))
teste.index = range(len(teste))

# Limpando a base de dados

In [5]:
tirar = ['!','@','#','$','%','¨','&','*','(',')','-','_','+','=','§',',','.','<','>',':',';','/','?','°','|','\\','{','}','ª','[',']','º','×','–','—','•','…','‹','›','€','1','2','3','4','5','6','7','8','9','0']
for i in tirar:
    treino.Email = treino.Email.str.replace(i, '', regex=False)
    teste.Email = teste.Email.str.replace(i, '', regex=False)

In [6]:
#CRIAR TABELA DE FREQUENCIA DE CADA PALAVRA
freq = {} #Tabela de frequências 
#Dicionario tabela de frequências:
    #   'H' = nº de vezes que determinada palavra aparece em email Ham
    #   'S' = nº de vezes que determinada palavra aparece em email Spam 
    #'pPiH' = P(P∩R) = prob de determinada palavra aparecer num email Ham
    #'pPiS' = P(P∩S) = prob de determinada palavra aparecer num email spam
    #'pPcH' = P(P|H) = prob de determinada palavra aparecer dado que o email é Ham
    #'pPcS' = P(P|S) = prob de determinada palavra aparecer dado que o email é Spam

for numLinha in range(len(treino.Email)): 
    palavras_linha = treino.loc[numLinha,'Email'].split(' ')
    for palavra in palavras_linha:
        if palavra not in freq:
            freq[palavra] = {'H':0,'S':0,'pPiH':0,'pPiS':0,'pPcH':0,'pPcS':0}
        if treino.loc[numLinha,'Class'] == 'ham':
            freq[palavra]['H'] +=1
        else:
            freq[palavra]['S'] +=1

In [84]:
#PH = Prob do email ser Ham | PS = Prob email ser Spam
PH = len(treino[treino['Class']=='ham'])/len(treino)
PS = len(treino[treino['Class']=='spam'])/len(treino)

palavrasH,palavrasS = 0,0 #total de palavras em emails HAM e total de palavras em emails Spam (contando repetições das palavras)
for key,valor in freq.items():  
    palavrasH += valor['H']
    palavrasS += valor['S']
totalPalavras = palavrasH + palavrasS

for palavra,valor in freq.items():
    #Prob de determinada palavra aparecer:
    pP = (valor['H']+valor['S'])/totalPalavras
    
    #adicionamos 1 ao numerador para que a probabilidade nunca dê zero
    freq[palavra]['pP'] = pP
    freq[palavra]['pPiH'] = (valor['H']+1)/(totalPalavras+1) #(2*totalR+totalI)
    freq[palavra]['pPiS'] = (valor['S']+1)/(totalPalavras+1) #/(2*totalI+totalR)
    
    freq[palavra]['pPcH'] = freq[palavra]['pPiH']/PH
    freq[palavra]['pPcS'] = freq[palavra]['pPiS']/PS

___
## 3. Classificador Naive-Bayes

In [86]:
mensagens={}

for numLinha in range(len(teste.Email)): 
    palavras_linha = teste.loc[numLinha,'Email'].split(' ')
    mensagens[numLinha] = {'Email':teste.loc[numLinha,'Email']}
    pLH,pLS =1,1    
    
    for palavra in palavras_linha:       
        if palavra in freq:
            pLH*= freq[palavra]['pPiH']/freq[palavra]['pP']
            pLS*= freq[palavra]['pPiS']/freq[palavra]['pP']
    
    mensagens[numLinha]['pLH'] = pLH
    mensagens[numLinha]['pLS'] = pLS
    
    if pLS > pLH:
        mensagens[numLinha]['Classe'] = 'spam'
    else:
        mensagens[numLinha]['Classe'] = 'ham'
        
        #E se a palavra não tiver na tabela de freq finge que ela n existe?

___
## 4. Qualidade do Classificador alterando a base de treinamento

In [88]:
VerRel, VerIre, FalIre, FalRel = 0,0,0,0
for numLinha in range(len(mensagens)):
    if (teste.loc[numLinha,'Class'] == 'ham')&(mensagens[numLinha]['Classe'] == 'ham'): #Verdadeiro relevante
        VerRel+=1
    if (teste.loc[numLinha,'Class'] == 'spam')&(mensagens[numLinha]['Classe'] == 'spam'): #Irrelevantes realmente irrelevantes
        VerIre+=1
    if (teste.loc[numLinha,'Class'] == 'spam')&(mensagens[numLinha]['Classe'] == 'ham'): #Relevantes que foram dados como irrelevantes
        FalIre+=1
    if (teste.loc[numLinha,'Class'] == 'ham')&(mensagens[numLinha]['Classe'] == 'spam'): #Irrelevantes que foram dados como relevantes
        FalRel+=1

print('Verdadeiros Relevantes : ',VerRel)
print('Verdadeiros Irrelevantes : ',VerIre)
print('Relevantes dados como irrelevantes: ',FalIre)
print('Irrelevantes dados como relevantes : ',FalRel)
print()
print('Precisão de acerto dos relevantes:',VerRel/VerRel+FalIre)
print('Precisão de acerto dos irrelevantes:',VerIre/VerIre+FalRel)

Verdadeiros Relevantes :  1195
Verdadeiros Irrelevantes :  121
Relevantes dados como irrelevantes:  76
Irrelevantes dados como relevantes :  1

Precisão de acerto dos relevantes: 77.0
Precisão de acerto dos irrelevantes: 2.0
